In [ ]:
import os
from uuid import uuid4
from dotenv import load_dotenv

from pinecone import Pinecone
from pinecone import ServerlessSpec

from langchain import hub
from langchain_core.documents import Document
from langchain_core.prompts import PromptTemplate
from langchain_pinecone import PineconeVectorStore
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_google_genai import GoogleGenerativeAIEmbeddings
load_dotenv()

True

In [63]:
os.environ['HF_TOKEN']=os.getenv("HF_TOKEN")
os.environ['GOOGLE_API_KEY']=os.getenv("GOOGLE_API_KEY")
os.environ['PINECONE_API_KEY']=os.getenv("PINECONE_API_KEY")


# embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

embeddings = GoogleGenerativeAIEmbeddings(model='models/embedding-001')
vector = embeddings.embed_query("hello, world!")

pc=Pinecone()

index_name="agenticbatch2"

if not pc.has_index(index_name):
    pc.create_index(
    name=index_name,
    dimension=768,
    metric="cosine",
    spec=ServerlessSpec(cloud="aws",region="us-east-1")    
)

#loading the index
index=pc.Index(index_name)
vector_store=PineconeVectorStore(index=index,embedding=embeddings)


# Creating the documents
document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},#additional info
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)


documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

#universal indentification number
uuids = [str(uuid4()) for _ in range(len(documents))]
vector_store.add_documents(documents=documents, ids=uuids)

retriever=vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.7} #hyperparameter
)


model=ChatGoogleGenerativeAI(model='gemini-1.5-flash')


prompt = hub.pull("rlm/rag-prompt")

prompt=PromptTemplate(
    template="""You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise.\nQuestion: {question} \nContext: {context} \nAnswer:""",
    input_variables=['context', 'question']
)

# prompt.invoke({"question":"what is a langchain?","context":"langchain is very super framework for LLM."})

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# --- [1] Interactive Query Loop for Notebooks ---
def run_rag_pipeline(question):
    # Get the relevant docs (for learning/debugging, not just answer)
    retrieved_docs = retriever.invoke(question)
    print(f"Retrieved {len(retrieved_docs)} documents for your query:")
    for i, doc in enumerate(retrieved_docs, 1):
        print(f"{i}. {doc.page_content}  [source: {doc.metadata.get('source','')}]")
    print("\nGenerating Answer...\n")
    answer = rag_chain.invoke(question)
    print("Answer:\n", answer)

In [64]:
run_rag_pipeline("what is langchain?")

Retrieved 4 documents for your query:
1. Building an exciting new project with LangChain - come check it out!  [source: tweet]
2. Building an exciting new project with LangChain - come check it out!  [source: tweet]
3. Building an exciting new project with LangChain - come check it out!  [source: tweet]
4. LangGraph is the best framework for building stateful, agentic applications!  [source: tweet]

Generating Answer...

Answer:
 Based on the provided text, LangChain appears to be a framework used for building projects.  The context suggests it's used to create exciting new applications.  More information is needed for a complete definition.


# first complete the remaining part of this notebook(create a proper rag)



# second assisgnment is: take a multiple pdf with text,image,table
1. fetch the data from pdf
2. at lesat there should be 200 pages
3. if chunking(use the sementic chunking technique) required do chunking and then embedding
4. store it inside the vector database(use any of them 1. mongodb 2. astradb 3. opensearch 4.milvus) ## i have not discuss then you need to explore
5. create a index with all three index machnism(Flat, HNSW, IVF) ## i have not discuss then you need to explore
6. create a retriever pipeline
7. check the retriever time(which one is fastet)
8. print the accuray score of every similarity search
9. perform the reranking either using BM25 or MMR ## i have not discuss then you need to explore
10. then write a prompt template
11. generte a oputput through llm
12. render that output over the DOCx ## i have not discuss then you need to explore
as a additional tip: you can follow rag playlist from my youtube

after completing it keep it on your github and share that link on my  mail id:
snshrivas3365@gmail.com

and share the assignment in your community chat as well by tagging krish and sunny

deadline is: till firday 9PM
   